In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'imagenet:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F341166%2F675926%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T212132Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1d64377b491c795b14dedce5e66df2cf27ea407046b50cb35223081ab816d9002a6e8a0b6e8ffb63d54dcfc2eceda6560cdef4c4c281472f7b7d0491370e7d97c056ae89177d00a15fb16cdb5ffb9f170ca45dc9f82a0c10efa0002024b4175085622e8add21115f585616743a54dfcf38599d5c4cb0932a2e9f7678ba03c04c73449426d5c5d777e03f4ffc9fc995fa9ab369964fb0c011e8e19168b22be969a3ffe488c65716dab6efe4088dbe3f3926b66623350ec4c083bfc806c4d2a7aeae673a7b49c4cc3689c624b34837c7d4159a32769ccf66300fb0ea26bb46bfa50ef6033ecec53b77d2d4c6ca8460eba5545d63521ea5be1fb66259bb810373a3,ham1000-segmentation-and-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1370616%2F2275763%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T212132Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2e872e8eebed2cf5f1720b1732e2870ed633bfdf7334cec077425c388123a0cb2ee456b8487eb8c57f656ec9cd7fd7426bf4404a2c17856fc3a6b4127f7990cbbc5750b742955d0266a5866c5321fa4fbe78e26f0920e6734facf339c7508efacb26d297a327bb980b0597e5504334ded7fafc9449c23fd2fa8249d59105d58a71d4c72c5fcb0678b7c5f128c9868553f3fcd283a17834f21014aab4e163143d8c2d58b9b819047bad74a7b354e9409ffe03fbf49129d314a8def593c33cfc3031b3e48c0266a1b48753fd97aac8f558a8d66e17aa2c7877104271915b53c59a90e947e3bbf106b0a1505b4c0a5173acc4153dd16fb030c19fcbca6e9abe576e,fitz17k-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4534408%2F7754808%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T212132Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e629f33b52ffa01962aebb78b8d81a586d8727ede88d7419f06b85c72868b82fea451b5167722ae4ae6668672951fd4cecd2db0899b265c349da7f77243888b9e340e18a9ba63728dfb7b298876580fea54d68d70af2d90459b92e3145219dce0809a2c480924911f60480ae30ed115c7526d951ed3a52db98dcc4d5d076ffffa5b73b9e52c4d79ec900764cb7ed53b7e6dd9f66fe9185dc4e30074be0d0804283c911608344712ebddf6b69ee63db1fefbb2626e863cee39ef4d7631a63d795f7c02d525927f2c9ca84dfe80f96e851f4bb0afb33dfd2ffad060fc0601f8731c6b0a2321b6cbb2f03b3ed0f03f6994a4655a4e04edda9168eeb10b3a0add62'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#Data Preprocessing
import numpy as np
import pandas as pd
from PIL import Image
import random
from imblearn.over_sampling import RandomOverSampler
import os
import shutil

def get_paths(directories):
    image_paths = []
    for direc in directories:
        for root, dirs, files in os.walk(direc):
            image_paths += [os.path.join(root, file) for file in files]
    return image_paths

def resize_and_save(input_path, output_path, new_size):
    original_image = Image.open(input_path)
    resized_image = original_image.resize(new_size)
    resized_image.save(output_path)

def Transform_data(Skin_image_directories, Non_skin_image_directories):

    skin_images = get_paths(Skin_image_directories)
    non_skin_images = get_paths(Non_skin_image_directories)

    labels = [0] * len(non_skin_images)
    labels += [1] * len(skin_images)
    image_paths = non_skin_images + skin_images

    oversample = RandomOverSampler(random_state = 3)
    image_paths, labels  = oversample.fit_resample(np.array(image_paths).reshape(-1, 1), labels)

    data = list (zip(image_paths.reshape(-1), labels) )

    random.seed(42)
    random.shuffle(data)

    print(f"Processing data Started..")
    num_images = len(data)
    new_size = (128, 128)

    i = 0
    for img, label in data:
        direc = '/kaggle/working/output/'+ str(label)
        if not os.path.exists(direc):
            os.makedirs(direc)
        img_path = img
        img_name = img_path.split('/')[-1]
        output_image_path = os.path.join(direc, img_name)

        if img_name.split('.')[1] == 'jpg' or img_name.split('.')[1] == 'JPEG':
            resize_and_save(img_path, output_image_path, new_size)
        i += 1
        if (i + 1) % (num_images // 10) == 0:
            print()
            print("Progress: " + str(i) + '/' + str(num_images)+ " "+ str(round(i /num_images * 100))+ "%")

Skin_image_directories  = ['/kaggle/input/ham1000-segmentation-and-classification/images/','/kaggle/input/fitz17k-dataset/data/finalfitz17k']
Non_skin_image_directories = ['/kaggle/input/imagenet/imagenet']
Transform_data(Skin_image_directories, Non_skin_image_directories)
shutil.make_archive('output', 'zip', '/kaggle/working/output')
print("\n Data Saved")